# Setting up experiment: which is faster, filter &rarr; join or join &rarr; filter

In [10]:
import random
random.seed(42)

list_len = 300000
key_list = [random.randrange(0, 100, 1) for _ in range(list_len)]
value_list = random.sample(range(list_len), list_len)

with open('text_files/filter_join_test/data_1.txt', 'w') as f:
    for i in range(len(key_list)):
        f.write(f'({key_list[i]}, {value_list[i]})\n')

In [11]:
random.seed(1)

list_len = 300000
key_list = [random.randrange(0, 100, 1) for _ in range(list_len)]
value_list = random.sample(range(list_len), list_len)

with open('text_files/filter_join_test/data_2.txt', 'w') as f:
    for i in range(len(key_list)):
        f.write(f'({key_list[i]}, {value_list[i]})\n')

In [26]:
def convert_to_tuple(str_data):
    key, value = str_data[1:-1].split(', ')
    return (key, int(value))

# Join &rarr; Filter

In [25]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf().setMaster('local[*]').setAppName('Filter_Join experiment')
spark_context = SparkContext().getOrCreate(spark_conf)

In [27]:
key_value_rdd_1 = spark_context.textFile('text_files/filter_join_test/data_1.txt').map(convert_to_tuple)
key_value_rdd_2 = spark_context.textFile('text_files/filter_join_test/data_2.txt').map(convert_to_tuple)

In [28]:
result_rdd_1 = key_value_rdd_1.join(key_value_rdd_2) \
                              .filter(lambda x: int(x[0]) >= 20)

In [29]:
result_rdd_1.repartition(12)

MapPartitionsRDD[15] at coalesce at NativeMethodAccessorImpl.java:0

In [30]:
result_rdd_1.count() # 2m 43.5s || 2m 39s

718924270

In [31]:
result_rdd_1.take(10)

[('86', (137034, 228832)),
 ('86', (137034, 67487)),
 ('86', (137034, 155298)),
 ('86', (137034, 218202)),
 ('86', (137034, 61189)),
 ('86', (137034, 271712)),
 ('86', (137034, 164849)),
 ('86', (137034, 122707)),
 ('86', (137034, 84715)),
 ('86', (137034, 41733))]

In [32]:
spark_context.stop()

# Filter &rarr; Join

In [33]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf().setMaster('local[*]').setAppName('Filter_Join experiment')
spark_context = SparkContext().getOrCreate(spark_conf)

In [34]:
key_value_rdd_1 = spark_context.textFile('text_files/filter_join_test/data_1.txt').map(convert_to_tuple)
key_value_rdd_2 = spark_context.textFile('text_files/filter_join_test/data_2.txt').map(convert_to_tuple)

In [35]:
result_rdd_2 = key_value_rdd_1.filter(lambda x: int(x[0]) >= 20) \
                              .join(key_value_rdd_2)

In [36]:
result_rdd_2.repartition(12)

MapPartitionsRDD[15] at coalesce at NativeMethodAccessorImpl.java:0

In [37]:
result_rdd_2.count() # 47s || 46.8s

718924270

In [38]:
result_rdd_2.take(10)

[('86', (137034, 228832)),
 ('86', (137034, 67487)),
 ('86', (137034, 155298)),
 ('86', (137034, 218202)),
 ('86', (137034, 61189)),
 ('86', (137034, 271712)),
 ('86', (137034, 164849)),
 ('86', (137034, 122707)),
 ('86', (137034, 84715)),
 ('86', (137034, 41733))]

In [39]:
spark_context.stop()